In [17]:
import pyodbc
import matplotlib.pyplot as plt
import io
import os
from PIL import Image
from numpy import asarray
import numpy as np
import cv2
from mtcnn.mtcnn import MTCNN
from numpy import expand_dims
from sklearn.ensemble import RandomForestClassifier
from tkinter import *
from tkinter import filedialog
import imageio
from PIL import ImageTk,Image
import tkinter.font as font
from playsound import playsound
from sklearn.metrics import plot_confusion_matrix


In [18]:

 
 
'''
conn = pyodbc.connect( 'Trusted_Connection=yes', Driver = "{SQL Server Native Client 11.0};" , Server= 'DESKTOP-E7I8FQH' , Database= 'Pictures')
 
 
 
d = "D:\FCAIH Material\Subjects Third Year\First Term\Artificial Intelligence\Project\Project 20 Code\images"
i=0
x=0
y=0
for path in os.listdir(d):
    if y==50:
        i=0
        x=0
    elif y > 50:
        if x == 5:
            i += 1
            x =0
    elif x==10:
        i += 1
        x = 0
    
    full_path = os.path.join(d, path)
    if os.path.isfile(full_path):
        cursor = conn.cursor()
        cursor.execute("insert into ImageTable (Class, image) values ("+str(i)+", (SELECT BulkColumn FROM Openrowset( Bulk '"+full_path+"', Single_Blob) as img))")
        cursor.commit()
    x +=1
    y +=1

'''

'\nconn = pyodbc.connect( \'Trusted_Connection=yes\', Driver = "{SQL Server Native Client 11.0};" , Server= \'DESKTOP-E7I8FQH\' , Database= \'Pictures\')\n \n \n \nd = "D:\\FCAIH Material\\Subjects Third Year\\First Term\\Artificial Intelligence\\Project\\images"\ni=0\nx=0\ny=0\nfor path in os.listdir(d):\n    if y==50:\n        i=0\n        x=0\n    elif y > 50:\n        if x == 5:\n            i += 1\n            x =0\n    elif x==10:\n        i += 1\n        x = 0\n    \n    full_path = os.path.join(d, path)\n    if os.path.isfile(full_path):\n        cursor = conn.cursor()\n        cursor.execute("insert into ImageTable (Class, image) values ("+str(i)+", (SELECT BulkColumn FROM Openrowset( Bulk \'"+full_path+"\', Single_Blob) as img))")\n        cursor.commit()\n    x +=1\n    y +=1\n\n'

In [19]:
#demonstrate face detection on 5 Celebrity Faces Dataset
# extract a single face from a given photograph
def extract_face(filename, required_size=(100,100)):
    # load image from file
    image = filename
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    # scale pixel values
    face_pixels = face_array.astype('float32')
    # normalize pixel values
    face_pixels = (face_pixels - np.min(face_pixels)) / (np.max(face_pixels) - np.min(face_pixels))
    face_pixels = cv2.cvtColor(face_pixels, cv2.COLOR_BGR2GRAY)
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    samples=samples.flatten()
    return samples

In [20]:
id=[]
images=[]
X_train=[]
y_train=[]
X_test=[]
y_test=[]

conn = pyodbc.connect( 'Trusted_Connection=yes', Driver = "{SQL Server Native Client 11.0};" , Server= 'DESKTOP-E7I8FQH' , Database= 'Pictures')



global img

cursor = conn.cursor()

cursor.execute("select * from ImageTable")

for row in cursor:
    id.append(row[1])
    images.append(row[2])

for i in range (0,50):
    image = Image.open(io.BytesIO(images[i]))
    face=extract_face(image).reshape(1,-1)
    X_train.append(face)
    y_train.append(id[i])

for i in range (50,len(images)):
    image = Image.open(io.BytesIO(images[i]))
    face=extract_face(image).reshape(1,-1)
    X_test.append(face)
    y_test.append(id[i])


In [11]:
image=Image.open('2.1.jpg')
face=extract_face(image).reshape(1,-1)

print(face.shape)


(1, 10000)


In [21]:
X_train = np.array(X_train)
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2])
y_train = np.array(y_train)
X_test = np.array(X_test)
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])
y_test = np.array(y_test)
#print(X_train.shape)
#print(X_test.shape)
#print(y_train.shape)
#print(y_test.shape)

(50, 1, 10000)
(50, 10000)


In [6]:
RandomForestClassifierModel = RandomForestClassifier(n_estimators = 200,max_depth=20,random_state=21) #criterion can be also : entropy 
RandomForestClassifierModel.fit(X_train, y_train)
y_pred = RandomForestClassifierModel.predict(X_test)
print('RandomForestClassifierModel Train Score is : ' , RandomForestClassifierModel.score(X_train, y_train)*100)
print('RandomForestClassifierModel Test Score is : ' , RandomForestClassifierModel.score(X_test, y_test)*100)

RandomForestClassifierModel Train Score is :  100.0
RandomForestClassifierModel Test Score is :  92.0


In [7]:
'''image = Image.open('47b50f681291eaac1d245d8c6aef08c4.jpg')
face=extract_face(image).reshape(1,-1)
print(face.shape)
y_pred = RandomForestClassifierModel.predict(face)
y_pred_prob = RandomForestClassifierModel.predict_proba(face)
if (y_pred==0):
    print('Mohamed Salah')
if (y_pred==1):
    print('Lionel Messi')
if (y_pred==2):
    print('Will Smith')
if (y_pred==3):
    print('Cristiano Ronaldo')
if (y_pred==4):
    print('Angelina Jolie')
print(y_pred_prob*100)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
imgplot = plt.imshow(face.reshape(100,100),cmap="gray")
plt.show()
'''

'image = Image.open(\'47b50f681291eaac1d245d8c6aef08c4.jpg\')\nface=extract_face(image).reshape(1,-1)\nprint(face.shape)\ny_pred = RandomForestClassifierModel.predict(face)\ny_pred_prob = RandomForestClassifierModel.predict_proba(face)\nif (y_pred==0):\n    print(\'Mohamed Salah\')\nif (y_pred==1):\n    print(\'Lionel Messi\')\nif (y_pred==2):\n    print(\'Will Smith\')\nif (y_pred==3):\n    print(\'Cristiano Ronaldo\')\nif (y_pred==4):\n    print(\'Angelina Jolie\')\nprint(y_pred_prob*100)\nimport matplotlib.pyplot as plt\nimport matplotlib.image as mpimg\nimgplot = plt.imshow(face.reshape(100,100),cmap="gray")\nplt.show()\n'

In [ ]:
root = Tk()
bg = ImageTk.PhotoImage(file="ground.jpg")
canvas = Canvas(root,width= 400, height= 300)
canvas.pack(fill= "both", expand=True)
canvas.create_image(0, 0, image=bg, anchor="nw")
root.title("Face Recognition Using Random Forest")
root.geometry("1000x702")
myFont1 = font.Font(family="Helvetica",size=12,weight="bold")
myFont2 = font.Font(family="Helvetica",size=18,weight="bold")

def browseImages():
    filename = filedialog.askopenfilename(initialdir = "/", title = "Select an Image",
                                          filetypes = (("jpg","*.jpg*"),
                                                       ("png","*.png*"),
                                                       ("jpeg","*.jpeg*"),
                                                       ("jfif","*.jfif*"),
                                                        )
                                         )
    global imag
    imag=Image.open(filename)
    img=ImageTk.PhotoImage(Image.open(filename).resize((350,350)))
    global label
    label = Label(root, padx=200 ,pady=150)
    label.place(relx = 0.5, rely = 0.42, anchor = 'center')
    label.configure(image=img)
    label.image = img

    
def classify_image(img):
    var = StringVar()
    face=extract_face(img).reshape(1,-1)
    y_pred = RandomForestClassifierModel.predict(face)
    global label3
    if(y_pred==0):
            label3 = Label(root,text="Mohamed Salah" , fg = "Black", padx=1000 ,pady=28,font=myFont2)
            label3.place(relx = 0.5, rely = 0.05,anchor='center')
            playsound("D:\FCAIH Material\Subjects Third Year\First Term\Artificial Intelligence\Project\project ai\sounds\Mo Salah, The Egyptian King! _ Learn LFC Songs (mp3cut.net).mp3")
    if(y_pred==1):
            label3 = Label(root,text="Lionel Messi" , fg = "Black", padx=1000 ,pady=28,font=myFont2)
            label3.place(relx = 0.5, rely = 0.05,anchor='center')
            playsound("D:\FCAIH Material\Subjects Third Year\First Term\Artificial Intelligence\Project\project ai\sounds\messi.mp3")
    if(y_pred==2):
            label3 = Label(root,text="Will Smith" , fg = "Black", padx=1000 ,pady=28,font=myFont2)
            label3.place(relx = 0.5, rely = 0.05,anchor='center')
            playsound("D:\FCAIH Material\Subjects Third Year\First Term\Artificial Intelligence\Project\project ai\sounds\mp3-output-ttsfree(dot)comwill.mp3")
    if(y_pred==3):
            label3 = Label(root,text="Cristiano Ronaldo" , fg = "Black", padx=1000 ,pady=28,font=myFont2)
            label3.place(relx = 0.5, rely = 0.05,anchor='center')
            playsound("D:\FCAIH Material\Subjects Third Year\First Term\Artificial Intelligence\Project\project ai\sounds\Siuuu Sound Effect #2.mp3")
    if(y_pred==4):
            label3 = Label(root,text="Angelina Jolie" , fg = "Black", padx=1000 ,pady=28,font=myFont2)
            label3.place(relx = 0.5, rely = 0.05,anchor='center')
            playsound("D:\FCAIH Material\Subjects Third Year\First Term\Artificial Intelligence\Project\project ai\sounds\mp3-output-ttsfree(dot)com.mp3")


def clear_label_image():
    label.destroy()
    label3.destroy()
    

    
def reco():
        classify_image(imag)
        


imageButton = Button(root, text="Browse Image", padx=20 ,pady=5, command=browseImages, fg="white",bg="#533535",font=myFont1,border=0)
imageButton.place(relx = 0.2, rely = 0.84, anchor = 'center')

recognizeButton = Button(root, text="Recognize !", padx=20 ,pady=5, command=reco,fg="white",bg="#064635",font=myFont1,border=0)
recognizeButton.place(relx = 0.51, rely = 0.88, anchor = 'center')

clearButton = Button(root, text="Clear image", padx=20 ,pady=5, command=clear_label_image,fg="white",bg="#533535",font=myFont1,border=0)
clearButton.place(relx = 0.8, rely = 0.84, anchor = 'center')



root.mainloop()

In [ ]:
from sklearn import metrics
y_pred = RandomForestClassifierModel.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_title('Random Forest Testing Performance');
plt.rcParams.update({'font.size': 15})
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(RandomForestClassifierModel, X_test, y_test ,ax=ax,cmap=plt.cm.Blues) 
plt.savefig('Random Forest Testing Performance.png')
plt.show()


fig, ax = plt.subplots(figsize=(10, 10))
ax.set_title('Random Forest Training Performance');
plt.rcParams.update({'font.size': 15})
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(RandomForestClassifierModel, X_train, y_train,ax=ax,cmap=plt.cm.Reds) 
plt.savefig('Random Forest Training Performance.png')
plt.show()